In [1]:
# !pip install openai

In [2]:
# !pip install --upgrade openai

In [21]:
import os
import openai
import time
import pandas as pd

In [1]:
openai.organization = ""#"X"
openai.api_key = "X"
openai.Model.list()

In [23]:
test_sample = pd.read_excel("D:/github/abstract-prediction/data_test.xlsx")

In [24]:
template = lambda t, a: f"""Title: {t}
Abstract: {a}
Based on the abstract/title, Is the abstract INCLUDE or EXCLUDE in our literature review? (answer "INCLUDE" or "EXCLUDE")"""
t = test_sample["Article Title"].iloc[0] 
a = test_sample["Abstract (AB)"].iloc[0]
prompt = template(t, a)
prompt

'Title: Does certified organic farming reduce greenhouse gas emissions from agricultural production?\nAbstract: The increasing prevalence of ecologically sustainable products in consumer markets, such as organic produce, are generally assumed to curtail anthropogenic impacts on the environment. Here I intend to present an alternative perspective on sustainable production by interpreting the relationship between recent rises in organic agriculture and greenhouse gas emissions from agricultural production. I construct two time series fixed-effects panel regressions to estimate how increases in organic farmland impact greenhouse gas emissions derived from agricultural production. My analysis finds that the rise of certified organic production in the United States is not correlated with declines in greenhouse gas emissions derived specifically from agricultural production, and on the contrary is associated positively overall agricultural greenhouse gas emissions. To make sense of this find

In [25]:
def chatgpt(message, text, temp=0.7, max_t=5):
    try: 
        output = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                  {"role": "system", "content": message},
                  {"role": "user", "content": text},
            ],
            temperature=temp,
            max_tokens=max_t,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0
        )
    except:
        print("Thinking...")
        time.sleep(1)
        return chatgpt(message, text, temp=temp, max_t=max_t)
    return output

In [26]:
%%time
system = "You are an expert on agroecology and impact on climate change. You have to decide which article is include or excluded in our literature review. "
chatgpt(system, prompt, temp=0.7, max_t=50)["choices"][0]["message"]["content"]

Wall time: 287 ms


'INCLUDE'

In [27]:
t, a, label = test_sample.sample(1).iloc[0]["Article Title  Abstract (AB)  Final decision".split("  ")]
prompt = template(t, a)
prompt, label

('Title: Influence of conservation tillage on Greenhouse gas fluxes and crop productivity in spring-wheat agroecosystems on the Loess Plateau of China\nAbstract: The effects of climate change such as dry spells, floods and erosion heavily impact agriculture especially smallholder systems on the Northwestern Loess Plateau of China. Nonetheless agriculture also contributes to global warming through the emission of greenhouse gases such as CO2, CH4 and N2O. Yet this complex conundrum can be alleviated and mitigated through sound soil and water management practices. Despite considerable literature on Conservation Agriculture (CA) as a strategy to improve the resilience and mitigation capacity of agroecosystems, there is still paucity of information on the impacts of CA on crop production and environmental quality on the Plateau. In order to fill this gap this study examined the effects of no-till and straw mulch on crop productivity and greenhouse gas fluxes in agroecosystems on the Platea

In [28]:
chatgpt(system, prompt, temp=0.7, max_t=50)["choices"][0]["message"]["content"]

'INCLUDE'

In [30]:
%%time
responses = []
for i, ix in enumerate(test_sample.index):
    t, a, label = test_sample.loc[ix]["Article Title  Abstract (AB)  Final decision".split("  ")]
    o = chatgpt(system, template(t, a), temp=0.7, max_t=50)["choices"][0]["message"]["content"]
    o = o.replace("\n", " ").strip()
    responses.append([ix, o])
    print(100*(i+1)/test_sample.shape[0], ix, o, label)

0.25 0 INCLUDE INCLUDE
0.5 1 INCLUDE EXCLUDE
0.75 2 INCLUDE INCLUDE
1.0 3 INCLUDE EXCLUDE
1.25 4 INCLUDE EXCLUDE
1.5 5 INCLUDE INCLUDE
1.75 6 INCLUDE EXCLUDE
2.0 7 INCLUDE EXCLUDE
2.25 8 INCLUDE EXCLUDE
2.5 9 INCLUDE INCLUDE
2.75 10 INCLUDE INCLUDE
3.0 11 INCLUDE INCLUDE
3.25 12 INCLUDE INCLUDE
3.5 13 INCLUDE EXCLUDE
3.75 14 INCLUDE INCLUDE
4.0 15 INCLUDE INCLUDE
4.25 16 INCLUDE EXCLUDE
4.5 17 INCLUDE INCLUDE
4.75 18 INCLUDE INCLUDE
5.0 19 INCLUDE INCLUDE
5.25 20 INCLUDE INCLUDE
5.5 21 INCLUDE INCLUDE
5.75 22 INCLUDE EXCLUDE
6.0 23 INCLUDE EXCLUDE
6.25 24 INCLUDE INCLUDE
6.5 25 INCLUDE EXCLUDE
6.75 26 INCLUDE EXCLUDE
7.0 27 INCLUDE INCLUDE
7.25 28 INCLUDE INCLUDE
7.5 29 INCLUDE EXCLUDE
7.75 30 INCLUDE INCLUDE
8.0 31 INCLUDE INCLUDE
8.25 32 INCLUDE EXCLUDE
8.5 33 INCLUDE EXCLUDE
8.75 34 INCLUDE INCLUDE
9.0 35 INCLUDE INCLUDE
9.25 36 INCLUDE INCLUDE
9.5 37 INCLUDE INCLUDE
9.75 38 INCLUDE EXCLUDE
10.0 39 INCLUDE EXCLUDE
10.25 40 INCLUDE EXCLUDE
10.5 41 INCLUDE INCLUDE
10.75 42 INCLUDE INC

In [31]:
test_pred = test_sample.copy()
test_pred["chatgpt"] = [x[1] for x in responses]
test_pred

,Unnamed: 0,Publication Type,Authors,Article Title,Abstract (AB),Publication Year,DOI,Researcher1,Researcher2,Researcher3,Final decision,Review studies,Duplicates,chatgpt
0,1,J,"McGee, JA",Does certified organic farming reduce greenhou...,The increasing prevalence of ecologically sust...,2015.0,10.1007/s10460-014-9543-1,INCLUDE,INCLUDE,NaN,INCLUDE,NaN,NaN,INCLUDE
1,3,J,"Chianese, DS; Rotz, CA; Richard, TL",SIMULATION OF NITROUS OXIDE EMISSIONS FROM DAI...,Farming practices can have a large impact on t...,2009.0,NaN,INCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,INCLUDE
2,21,J,"Lehuger, S; Gabrielle, B; Laville, P; Lamboni,...",Predicting and mitigating the net greenhouse g...,"Nitrous oxide, carbon dioxide and methane are ...",2011.0,10.1016/j.agrformet.2011.07.002,INCLUDE,INCLUDE,NaN,INCLUDE,NaN,NaN,INCLUDE
3,27,J,"Kahrl, F; Li, YJ; Su, YF; Tennigkeit, T; Wilke...",Greenhouse gas emissions from nitrogen fertili...,The use of synthetic nitrogen (N) fertilizers ...,2010.0,10.1016/j.envsci.2010.07.006,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,INCLUDE
4,31,J,"Schneider, UA; McCarl, BA",Economic potential of biomass based fuels for ...,Use of biofuels diminishes fossil fuel combust...,2003.0,10.1023/A:1023632309097,EXCLUDE,EXCLUDE,NaN,EXCLUDE,NaN,NaN,INCLUDE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,1988,J,"Laamrani, A; Voroney, PR; Berg, AA; Gillespie,...",Temporal Change of Soil Carbon on a Long-Term ...,The impacts of tillage practices and crop rota...,2020.0,10.3390/agronomy10060840,INCLUDE,NaN,INCLUDE,INCLUDE,NaN,NaN,INCLUDE.
396,1994,J,"Vijayakumar, S; Bazrgar, AB; Coleman, B; Gordo...",Carbon stocks in riparian buffer systems at si...,"In any given watershed, riparian buffer system...",2020.0,10.1016/j.agee.2020.107149,INCLUDE,INCLUDE,EXCLUDE,EXCLUDE,NaN,NaN,INCLUDE
397,1995,J,"Edberg, S; Tigabu, M; Oden, PC",Commercial Eucalyptus Plantations with Taungya...,"The increasing demand for wood, fiber, and pul...",2022.0,10.3390/f13091395,INCLUDE,NaN,INCLUDE,INCLUDE,NaN,NaN,INCLUDE
398,1997,J,"Kollah, B; Dubey, G; Parasai, P; Saha, JK; Gan...",Interactive effect of biochar size and organic...,Biochar (BC) application to agricultural soil ...,2015.0,10.1111/sum.12168,EXCLUDE,NaN,EXCLUDE,EXCLUDE,NaN,NaN,INCLUDE


In [32]:
test_pred.to_excel("chatgpt_responses_test_pred_s0_v2.xlsx")

In [33]:
from sklearn.metrics import classification_report

In [34]:
print(classification_report(test_pred["Final decision"].apply(lambda x: int("include" in x.lower())), test_pred["chatgpt"].apply(lambda x: int("include" in x.lower())), digits=4))

              precision    recall  f1-score   support

           0     1.0000    0.0053    0.0106       188
           1     0.5313    1.0000    0.6939       212

    accuracy                         0.5325       400
   macro avg     0.7657    0.5027    0.3523       400
weighted avg     0.7516    0.5325    0.3728       400

